In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
import numpy as np

In [ ]:
# URL of different Tennessee Districts
#URL = ['https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2022&id=NH01&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=IL05&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2022&id=AL01&spec=N'


#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
    #'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N'
 #'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N'
# 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N'
#  'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N'   
#  'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N'
#  'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N'
#'https://www.opensecrets.org/races/candidates?cycle=2022&id=AL01&spec=N'

#response = requests.get(URL)
#soup = BS(response.content, 'html.parser')

In [2]:

#Creating empty dataframe with column names
Election = pd.DataFrame(columns = ['Name',
                                   'Party',
                                   'State', 
                                   'District_Number',
                                   'Incumbent',
                                   'Result',
                                   'Vote_Percentage',
                                   'Amount_Raised ($)',
                                   'Amount_Spent ($)'])
#Election

# Function to URL links
def StateDistrictLinks(URL, State):
    DistrictLinks = []
    baseURL = URL[0: URL.index("/", 9)]
    page = requests.get(URL)
    soupS= BS(page.content, "html.parser")

    # find all districts page links in a state
    for idx, subNavLinks in enumerate(soupS.find_all("a", class_="SubNav-link")):
        print(subNavLinks)
        res= re.findall("id=\w{2}\d+", subNavLinks["href"])
        if(res) :
            DistrictLinks.append(baseURL + subNavLinks["href"].replace("summary?","candidates?"))
    
    return DistrictLinks
    
def election2020(URL):
    response = requests.get(URL)
    soup = BS(response.content, 'html.parser')
   
    global Election
    
# List of variables 
    Candidate_Name = ""
    Candidate_Party = ""
    Candidate_State =""
    Candidate_District =""
    C_Incumbent = ""
    Won_Race =""
    Vote_Percentage = np.NaN
    Amount_Raised = np.NaN
    Amount_Spent = np.NaN

# Loop to find
    for amount in soup.find_all("div", class_= "Members--list-item"):
        #print(amount)
    
        for member in amount.find_all("div", class_= "Members--bio u-richtext"):
            Candidate = member.find('strong').text
            #print(Candidate)
            Candidate_Party = re.findall("\(\w\)", Candidate)[0].replace('(', '').replace(')', '') 
            Candidate_Name = Candidate.split(' (')[0].replace("\t", "").replace("\n", "")
            #print(Candidate_Name)
            Candidate_Party = re.findall("\(\w\)", Candidate)[0].replace('(', '').replace(')', '') 
            #print(Candidate_Party)
            #Vote= re.findall("(\d+.\d%)", Candidate)[0].replace('(', '').replace('%',' ').replace(')', '') 
            #print(Vote)
        
            if "Incumbent" in Candidate:
                C_Incumbent = True
                #print(C_Incumbent)
            else: 
                C_Incumbent = False
            
            if "Winner" in Candidate:
                Won_Race = True
                #print(Won_Race)
            else:
                Won_Race = False
        
            if "vote" in Candidate:
                Vote= re.findall("(\d+.\d%)", Candidate)[0].replace('(', '').replace('%',' ').replace(')', '') 
                #print(Vote)
            else: 
                Vote = ' '
        
            for title in soup.find_all('h1', class_= "Hero-title"):
                #print(title)
                Candidate_State = title.text.split(' District')[0]
                #print(Candidate_State)
                Candidate_District = title.text.split('District ')[1].split(" ")[0]
                #print(Candidate_District) 
                     
                raised = amount.find_all("td", class_= "Members--number")[0].text.replace('$',"").replace(",", "")
                #print(raised)
                spent = amount.find_all("td", class_= "Members--number")[1].text.replace('$',"").replace(",", "")
                #print(spent)
                Election = Election.append({'Name': Candidate_Name, 'Party': Candidate_Party, 'State' : Candidate_State, 
                                            'District_Number': Candidate_District ,'Incumbent': C_Incumbent, 
                                            'Result': Won_Race, 'Vote_Percentage': Vote, 'Amount_Raised ($)': raised, 
                                            'Amount_Spent ($)': spent}, ignore_index = True) 
        

In [3]:
State_abbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA',
'WA','WV','WI','WY']

print(State_abbr)

['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [18]:
StateRepURL = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
response = requests.get(StateRepURL)
soup = BS(response.content, 'html.parser')
StateRepList = soup.find_all("div", class_="md-drag md-table-wrapper")[0].find_all('tr')
#print(StateRepList)

StateRep = pd.DataFrame(columns = ['State', 
                                   'State_Abbr',
                                   'Representatives'])
for State in StateRepList:
    #print(State)
    State_Name = ""
    Reps = "" 
    State_Name = State.find_all("td", attrs={'headers' : 's'})
    #print(State_Name)
    Reps = State.find_all("td", attrs={'headers' : 'r'})
    #print(Reps)
    if( len(State_Name) > 0) :
        State_Name = State_Name[0].text.replace("\n","").strip()
    if( len(Reps) > 0) :
        Reps = Reps[0].text.replace("\n","").strip()    
    StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
StateRep = StateRep.drop([0,51])
StateRep ['State_Abbr']= State_abbr
#StateRep


def StateDistrictLinks(URL, State):#,Representative):
    DistrictLinks = []
    baseURL = URL[0: URL.index("/")]
    page = requests.get(URL)
    soupS= BS(page.content, "html.parser")

    # find all districts page links in a state
    for idx, subNavLinks in enumerate(soupS.find_all("a", class_="SubNav-link")):
        print(subNavLinks)
        res= re.findall("id=\w{2}\d+", subNavLinks["href"])
        if(res) :
            DistrictLinks.append(baseURL + subNavLinks["href"].replace("summary?","candidates?"))
        
        
    #return DistrictLinks

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69551/363006996.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69551/363006996.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69551/363006996.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69

/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69551/363006996.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69551/363006996.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69551/363006996.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StateRep = StateRep.append({'State' : State_Name , "Representatives":Reps},ignore_index = True)
/var/folders/s3/9_r9v9px7_v0q69b5zcbnmsm0000gn/T/ipykernel_69

In [7]:
StateRep

,State,State_Abbr,Representatives
1,Alabama,AL,7
2,Alaska,AK,1
3,Arizona,AZ,9
4,Arkansas,AR,4
5,California,CA,53
6,Colorado,CO,7
7,Connecticut,CT,5
8,Delaware,DE,1
9,Florida,FL,27
10,Georgia,GA,14


In [9]:
 StateDistrictLinks("https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N'","Alabama", "7")

<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=ALS1&amp;spec=N">
                        Alabama Senate <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL01&amp;spec=N">
                        Alabama District 01 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL02&amp;spec=N">
                        Alabama District 02 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL03&amp;spec=N">
                        Alabama District 03 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL04&amp;spec=N">
                        Alabama District 04 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL05&amp;spec=N">
               

['https:/races/candidates?cycle=2020&id=AL01&spec=N',
 'https:/races/candidates?cycle=2020&id=AL02&spec=N',
 'https:/races/candidates?cycle=2020&id=AL03&spec=N',
 'https:/races/candidates?cycle=2020&id=AL04&spec=N',
 'https:/races/candidates?cycle=2020&id=AL05&spec=N',
 'https:/races/candidates?cycle=2020&id=AL06&spec=N',
 'https:/races/candidates?cycle=2020&id=AL07&spec=N']

In [23]:
URLStateDisList= "https://www.opensecrets.org/races/candidates?cycle=2020&id=STATE_abbr01&spec=N"

USStateDistLinks= []
for index, row in StateRep.iterrows():
    #print (index,row["State"], row["State_Abbr"])
    strlnk = URLStateDisList.replace("STATE_abbr",row["State_Abbr"])
    USStateDistLinks.append(strlnk)
    linksDistricts = StateDistrictLinks(strlnk, "State")
    #print(linksDistricts)
    
    for link in linksDistricts:
        print(link)
        USStateDistLinks.append(link)
        election2020(link)
        #print(USStateDistLinks)

<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=ALS1&amp;spec=N">
                        Alabama Senate <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link SubNav-link--active" href="/races/summary?cycle=2020&amp;id=AL01&amp;spec=N">
                        Alabama District 01 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL02&amp;spec=N">
                        Alabama District 02 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL03&amp;spec=N">
                        Alabama District 03 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL04&amp;spec=N">
                        Alabama District 04 <i aria-hidden="true" class="fa fa-angle-right"></i>
</a>
<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=AL05&amp;spec

TypeError: 'NoneType' object is not iterable